# Predicción del set de Fine Food Reviews con PPMC

Usamos PPMC para entrenar 5 objetos PPMC distintos, uno para cada puntaje. Luego para predecir cada review del test lo comprimimos con cada uno de los 5 PPMCs y nos quedamos con el label del que comprima mejor.

Elegimos PPMC de orden 4 porque por experiencia con Vowpal Wabbit resultó ser el mejor n-grama.

In [1]:
PPMC_ORDER = 4

# Campos dentro del CSV
ID = 0
TEXT = 9
PREDICTION = 6

In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import csv
from PPMC import PPMC

In [3]:
# Función que sirve cuando el RDD es (Predicción, (PPMC, [Text1,Text2,...,TextN]))
def compress_all(line):
    p = line[1][0]
    reviews = line[1][1]
    for review in reviews:
        p.compress(review)
    return (line[0], (p, len(reviews)))

Operamos dentro del RDD para comprimir todas las reviews de un mismo puntaje con el mismo objeto PPMC.
De esta forma, cuando terminemos vamos a tener 5 PPMCs, uno por cada puntaje.

In [4]:
# Montamos el CSV a un RDD, cada registro es una linea del archivo
trainRDD = sc.textFile("train_sin_repeticiones.csv", minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))

# Extraemos la cabecera
trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

# Tomamos una porción del RDD para probar
# trainRDD = trainRDD.sample(False, 0.01)

# (Predicción,[Texto])
trainRDD = trainRDD.map(lambda line: (line[PREDICTION], [line[TEXT]]))

# (Predicción, [Text1,Text2,...,TextN])
trainRDD = trainRDD.reduceByKey(lambda value1, value2: value1+value2)

# (Predicción, (PPMC, [Text1,Text2,...,TextN]))
trainRDD = trainRDD.map(lambda line: (line[0], (PPMC(PPMC_ORDER), line[1])))

# (Predicción, (PPMC, Cantidad de reviews))
trainRDD = trainRDD.map(lambda line: compress_all(line))

A continuación, con los PPMCs ya entrenados, procedemos a comprimir cada review del test con cada uno de los 5 PPMCs. El que nos devuelve el número de menor cantidad de bits es el que mejor comprime. Esto significa que está más cerca de las reviews comprimidas con ese PPMC, por lo tanto se lo clasifica con ese label.

In [5]:
import copy

# Montamos el CSV a un RDD, cada registro es una linea del archivo
testRDD = sc.textFile("test_limpio.csv", minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))

# Extraemos la cabecera
testRDD = testRDD.filter(lambda line: line[ID] != "Id")

# Tomamos una porción del RDD para probar
# testRDD = testRDD.sample(False, 0.01)

# (Id, Texto)
resultRDD = testRDD.map(lambda line: (line[ID], line[TEXT-1]))

# ((Id, Testo),(Predicción, (PPMC, Cantidad de reviews)))
resultRDD = resultRDD.cartesian(trainRDD)

In [6]:
# (Id, (Predicción, Cantidad de bits, Cantidad de reviews))
resultRDD = resultRDD.map(lambda line: (line[0][0], (line[1][0], len(line[1][1][0].copy().compress(line[0][1])), 
                                         line[1][1][1])))

# (Id, (Predicción, Cantidad de bits, Cantidad de reviews))
resultRDD = resultRDD.reduceByKey(lambda value1, value2: value1 if value1[1] < value2[1] else value2)

# (Id, Predicción)
resultRDD = resultRDD.map(lambda line: (line[0], line[1][0]))

Por último generamos el archivo de salida con las predicciones realizadas.

In [ ]:
# Archivo de salida

lines = resultRDD.map(lambda line: ",".join(str(d) for d in line))
lines.saveAsTextFile("test_prediction_ppmc.csv")
